In [33]:
import pandas as pd
import numpy as np

gicc05 = pd.read_csv('files/GICC05 data/datasets/GICC05_time_scale.tab', skiprows=40, delimiter='\t')

gicc05 = gicc05.iloc[:, [0,2,3,6]]

gicc05 = gicc05[gicc05['Age [a] (b2k)'] > 3244]

gicc05['comb_ngrip'] = np.where(
    gicc05['Depth ice/snow [m] (NGRIP1)'] < 1346.42,
    gicc05['Depth ice/snow [m] (NGRIP1)'],
    gicc05['Depth ice/snow [m] (NGRIP2)']
)

gicc05['comb_ngrip']

#gicc05.to_csv('files/GICC05 data/datasets/GICC05_combined.csv', index=False)

3241      580.810
3242      580.910
3243      581.060
3244      581.200
3245      581.350
           ...   
60194    2428.758
60195    2428.768
60196    2428.776
60197    2428.784
60198    2428.793
Name: comb_ngrip, Length: 56958, dtype: float64

In [34]:
fullngrip_combined = pd.read_excel('files/fullNGRIP_combined.xlsx',skiprows=2, header=None, comment="#", names=["depth", "age", "sigma"])

fullngrip_combined

,depth,age,sigma
0,9.900,27,2.0
1,10.257,28,2.0
2,10.647,29,2.0
3,10.919,30,2.0
4,11.226,31,2.0
...,...,...,...
60171,2428.758,60198,2611.0
60172,2428.768,60199,2611.0
60173,2428.776,60200,2611.0
60174,2428.784,60201,2611.0


In [35]:

#df = pd.read_csv(agedepth_file, delimiter="\t", comment="#", names=["depth", "age", "sigma"])

df = fullngrip_combined.copy(deep=True)

# Define year intervals
min_age = df["age"].min()
max_age = df["age"].max()
intervals = np.arange(min_age, max_age, 100) #set year intervals, ignoring the end if less than 100 years

depths = np.interp(intervals, df["age"], df["depth"])

sigma_start = np.interp(intervals, df["age"], df["sigma"])
sigma_end = np.interp(intervals+99, df["age"], df["sigma"])

sigma_diff = sigma_end - sigma_start

#this is here to test for an error
for i in range(len(sigma_diff)):
    if sigma_diff[i] == 0:
        sigma_diff[i] += 0.01

interval_df = pd.DataFrame({
    "#top depth (m)": depths[:-1],
    "bottom depth (m)": depths[1:],
    "duration (yr)": 100,
    "sigma duration (yr)": sigma_diff[:-1]
})

interval_df.dropna(inplace=True)

output_file = "ice_age_intervals.txt"

interval_df.to_csv(output_file, sep="\t", index=False)

print(f"Done! Saved to: {output_file}")

Done! Saved to: ice_age_intervals.txt
